In [16]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

In [17]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console

In [18]:
model = OpenAIChatCompletionClient(model="gpt-4o-mini", api_key=api_key)

clarity_agent = AssistantAgent(
    "ClarityAgent",
    model_client=model,
    system_message="""You are an expert editor focused on clarity and simplicity. 
            Your job is to eliminate ambiguity, redundancy, and make every sentence crisp and clear. 
            Don't worry about persuasion or tone — just make the message easy to read and understand.""",
)

tone_agent = AssistantAgent(
    "ToneAgent",
    model_client=model,
    system_message="""You are a communication coach focused on emotional tone and professionalism. 
            Your job is to make the email sound warm, confident, and human — while staying professional 
            and appropriate for the audience. Improve the emotional resonance, polish the phrasing, 
            and adjust any words that may come off as stiff, cold, or overly casual.""",
)

persuasion_agent = AssistantAgent(
    "PersuasionAgent",
    model_client=model,
    system_message="""You are a persuasion expert trained in marketing, behavioral psychology, 
            and copywriting. Your job is to enhance the email's persuasive power: improve call to action, structure arguments, and emphasize benefits. Remove weak or passive language.""",
)

synthesizer_agent = AssistantAgent(
    "SynthesizerAgent",
    model_client=model,
    system_message="""You are an advanced email-writing specialist. Your role is to read all 
            prior agent responses and revisions, and then **synthesize the best ideas** into a unified, 
            polished draft of the email. Focus on: Integrating clarity, tone, and persuasion improvements; 
            Ensuring coherence, fluency, and a natural voice; Creating a version that feels professional, 
            effective, and readable.""",
)

critic_agent = AssistantAgent(
    "CriticAgent",
    model_client=model,
    system_message="""You are an email quality evaluator. Your job is to perform a final review 
            of the synthesized email and determine if it meets professional standards. Review the email for: 
            Clarity and flow, appropriate professional tone, effective call-to-action, and overall coherence.
            Be constructive but decisive. If the email has major flaws (unclear message, unprofessional tone, 
            or missing key elements), provide ONE specific improvement suggestion. If the email meets professional standards and communicates effectively, respond with 'The email meets professional standards.' followed by `TERMINATE` on a new line. You should only approve emails ar good enough for professional use, even if not perfect.""",
)

In [19]:
text_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=30)

termination_condition = text_termination | max_messages_termination

In [20]:
team = RoundRobinGroupChat(
    participants=[
        clarity_agent,
        tone_agent,
        persuasion_agent,
        synthesizer_agent,
        critic_agent,
    ],
    termination_condition=termination_condition,
)

await Console(
    team.run_stream(
        task="Hi! Im hungry, buy me lunch and invest in my business. Thanks."
    )
)

---------- TextMessage (user) ----------
Hi! Im hungry, buy me lunch and invest in my business. Thanks.


---------- TextMessage (ClarityAgent) ----------
Hello! I'm hungry. Please buy me lunch and invest in my business. Thank you.
---------- TextMessage (ToneAgent) ----------
Subject: A Quick Favor 

Hi there,

I hope this message finds you well! I’m feeling a bit hungry today and was wondering if you might be willing to treat me to lunch. Additionally, I would greatly appreciate any support you could offer as I look to invest in my business.

Thank you so much for considering my requests! I truly value your support.

Best regards,  
[Your Name]  
---------- TextMessage (PersuasionAgent) ----------
Subject: A Special Request to Fuel My Passion

Hi [Recipient's Name],

I hope this message finds you thriving! I’m reaching out with a couple of requests that could make a significant difference for me.

First, I’d love your help with lunch today. It’s not just about the meal—I believe that sharing a meal fosters connection and collaboration. It would mean a lot to me to enjoy some quality time

TaskResult(messages=[TextMessage(id='a19774b5-61c0-4969-9411-7df91273fdd9', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 17, 6, 23, 23, 835383, tzinfo=datetime.timezone.utc), content='Hi! Im hungry, buy me lunch and invest in my business. Thanks.', type='TextMessage'), TextMessage(id='8ef27c40-cdca-490e-bc19-0f711f5ba6c9', source='ClarityAgent', models_usage=RequestUsage(prompt_tokens=77, completion_tokens=18), metadata={}, created_at=datetime.datetime(2025, 9, 17, 6, 23, 24, 988709, tzinfo=datetime.timezone.utc), content="Hello! I'm hungry. Please buy me lunch and invest in my business. Thank you.", type='TextMessage'), TextMessage(id='cfd1fe55-5584-4786-ae5b-8bd29995d336', source='ToneAgent', models_usage=RequestUsage(prompt_tokens=123, completion_tokens=82), metadata={}, created_at=datetime.datetime(2025, 9, 17, 6, 23, 27, 104027, tzinfo=datetime.timezone.utc), content='Subject: A Quick Favor \n\nHi there,\n\nI hope this message finds you well